In [1]:
!pip install transformers
!pip install datasets
!pip install tiktoken
!pip install wandb
!pip install tqdm

  Using cached transformers-4.26.1-py3-none-any.whl (6.3 MB)
  Using cached tokenizers-0.13.2-cp39-cp39-macosx_12_0_arm64.whl (3.7 MB)
  Using cached wandb-0.13.10-py3-none-any.whl (2.0 MB)
  Using cached GitPython-3.1.31-py3-none-any.whl (184 kB)
  Using cached setproctitle-1.3.2-cp39-cp39-macosx_10_9_universal2.whl (16 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached protobuf-4.22.0-cp37-abi3-macosx_10_9_universal2.whl (397 kB)
  Using cached pathtools-0.1.2.tar.gz (11 kB)
  Preparing metadata (setup.py) ... done
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached sentry_sdk-1.16.0-py2.py3-none-any.whl (184 kB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=bd4e28b16e1a5edef3b318e33e9364516f700a9a87288172de2d7d1e2288b98d
  Stored 

In [1]:
import pandas as pd
data = pd.read_json('data/mtg_char/default-cards-20230313210730.json')

In [5]:
# peek at the data
print(data.shape)
data.head()

(80731, 84)


,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,tcgplayer_etched_id,attraction_lights,color_indicator,variation_of,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,card,0000a54c-a511-4925-92dc-01b937f9afad,dc4e2134-f0c2-49aa-9ea3-ebf83af1445c,[],NaN,NaN,98659.0,NaN,Spirit,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,65171.0,145764.0,301766.0,Siren Lookout,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],NaN,NaN,1623.0,5664.0,Web,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# drop some unnecessary columns
data.drop(columns=['mtgo_id', 'mtgo_foil_id', 'arena_id', 'tcgplayer_id', 'cardmarket_id', 'legalities', 'games', 'reserved', 'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint', 'variation', 'artist', 'artist_ids', 'illustration_id', 'border_color', 'booster', 'story_spotlight', 'edhrec_rank', 'penny_rank','prices', 'promo_types', 'arena_id', 'preview', 'security_stamp', 'tcgplayer_etched_id', 'variation_of'])

,object,id,oracle_id,multiverse_ids,name,lang,released_at,uri,scryfall_uri,layout,...,printed_name,card_faces,attraction_lights,color_indicator,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],Fury Sliver,en,2006-10-06,https://api.scryfall.com/cards/0000579f-7b35-4...,https://scryfall.com/card/tsp/157/fury-sliver?...,normal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],Kor Outfitter,en,2009-10-02,https://api.scryfall.com/cards/00006596-1166-4...,https://scryfall.com/card/zen/21/kor-outfitter...,normal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,card,0000a54c-a511-4925-92dc-01b937f9afad,dc4e2134-f0c2-49aa-9ea3-ebf83af1445c,[],Spirit,en,2015-05-22,https://api.scryfall.com/cards/0000a54c-a511-4...,https://scryfall.com/card/tmm2/5/spirit?utm_so...,token,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],Siren Lookout,en,2017-09-29,https://api.scryfall.com/cards/0000cd57-91fe-4...,https://scryfall.com/card/xln/78/siren-lookout...,normal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],Web,en,1994-04-01,https://api.scryfall.com/cards/00012bd8-ed68-4...,https://scryfall.com/card/3ed/229/web?utm_sour...,normal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80726,card,fffc0bfd-8e64-44bf-ae0a-5d2ee54c58df,dc4a4996-108a-4aac-850f-2d9f76403446,[],Boomerang,zhs,1997-03-24,https://api.scryfall.com/cards/fffc0bfd-8e64-4...,https://scryfall.com/card/5ed/75s/zhs/boomeran...,normal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80727,card,fffcbc4e-c6dc-4808-b262-f7c453e74dd8,2979597d-c3cc-4585-a9b7-7fa15641b030,[],Celestine Reef,en,2009-09-30,https://api.scryfall.com/cards/fffcbc4e-c6dc-4...,https://scryfall.com/card/pdci/42/celestine-re...,planar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80728,card,fffce2f7-b619-4483-a75e-916343194641,6ecc61ff-719a-405d-b02a-01607729d05c,[45404],Horned Troll,en,2003-07-28,https://api.scryfall.com/cards/fffce2f7-b619-4...,https://scryfall.com/card/8ed/257/horned-troll...,normal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80729,card,fffe7b2b-22c3-4e6a-9b1b-c6d7b29b9f86,d7ebd4a7-c542-469f-8a1c-d48b92fa540a,[439664],Exultant Skymarcher,en,2018-01-19,https://api.scryfall.com/cards/fffe7b2b-22c3-4...,https://scryfall.com/card/rix/7/exultant-skyma...,normal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# remove some more columns
data.drop(columns=['released_at', 'set_id','set', 'set_uri', 'set_search_uri', 'scryfall_set_uri', 'rulings_uri', 'prints_search_uri', 'card_back_id', 'frame', 'related_uris'])

,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,tcgplayer_etched_id,attraction_lights,color_indicator,variation_of,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,65171.0,145764.0,301766.0,Siren Lookout,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],NaN,NaN,1623.0,5664.0,Web,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,card,0001f1ef-b957-4a55-b47f-14839cdbab6f,ef027846-be81-4959-a6b5-56bd01b1e68a,[472997],78170.0,NaN,198861.0,400134.0,Venerable Knight,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80690,card,ffdfad1f-d316-4023-9c79-8c24f1cd31d0,7ddd84d2-1a6d-43f5-a42d-3c66eee5f7e5,[548297],96918.0,NaN,262429.0,607507.0,Born to Drive,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80700,card,ffe4d34f-68f0-4d79-9aab-58c5304224d9,6dab0e1a-f39a-4113-9d13-b067cfefbdb3,[240069],43977.0,43978.0,58898.0,254643.0,Stern Mentor,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80703,card,ffe93b27-f8ae-4abf-8ade-90f503f132c2,6d37ba4b-ff56-4eec-9dc2-2d7f357dc9c9,[506924],87287.0,NaN,229263.0,527575.0,Youthful Valkyrie,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80704,card,ffe9ee1c-5eb3-4d63-a641-0ec5adf7b058,e60b23ba-2258-4da1-b873-452ee215846b,[583793],105040.0,NaN,452220.0,682620.0,Fallaji Vanguard,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# filter out cards that are not in english
data = data[data['lang'] == 'en']
print(data.shape)
data.head()

(78312, 84)


,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,tcgplayer_etched_id,attraction_lights,color_indicator,variation_of,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,card,0000a54c-a511-4925-92dc-01b937f9afad,dc4e2134-f0c2-49aa-9ea3-ebf83af1445c,[],NaN,NaN,98659.0,NaN,Spirit,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,65171.0,145764.0,301766.0,Siren Lookout,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],NaN,NaN,1623.0,5664.0,Web,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# a function to print out the shape and head of a dataframe
def print_df(df):
    print(df.shape)
    df.head()

In [10]:
# Keep only normal cards
data = data[data['layout'] == 'normal']
print_df(data)

(73145, 84)
  object                                    id  \
0   card  0000579f-7b35-4ed3-b44c-db2a538066fe   
1   card  00006596-1166-4a79-8443-ca9f82e6db4e   
3   card  0000cd57-91fe-411f-b798-646e965eec37   
4   card  00012bd8-ed68-4978-a22d-f450c8a6e048   
5   card  0001f1ef-b957-4a55-b47f-14839cdbab6f   

                              oracle_id multiverse_ids  mtgo_id  mtgo_foil_id  \
0  44623693-51d6-49ad-8cd7-140505caf02f       [109722]  25527.0       25528.0   
1  8ae3562f-28b7-4462-96ed-be0cf7052ccc       [189637]  34586.0       34587.0   
3  9f0d82ae-38bf-45d8-8cda-982b6ead1d72       [435231]  65170.0       65171.0   
4  5aa12aff-db3c-4be5-822b-3afdf536b33e         [1278]      NaN           NaN   
5  ef027846-be81-4959-a6b5-56bd01b1e68a       [472997]  78170.0           NaN   

   tcgplayer_id  cardmarket_id              name lang  ...  \
0       14240.0        13850.0       Fury Sliver   en  ...   
1       33347.0        21851.0     Kor Outfitter   en  ...   
3      145764.

In [11]:
print_df(data)

(73145, 84)
  object                                    id  \
0   card  0000579f-7b35-4ed3-b44c-db2a538066fe   
1   card  00006596-1166-4a79-8443-ca9f82e6db4e   
3   card  0000cd57-91fe-411f-b798-646e965eec37   
4   card  00012bd8-ed68-4978-a22d-f450c8a6e048   
5   card  0001f1ef-b957-4a55-b47f-14839cdbab6f   

                              oracle_id multiverse_ids  mtgo_id  mtgo_foil_id  \
0  44623693-51d6-49ad-8cd7-140505caf02f       [109722]  25527.0       25528.0   
1  8ae3562f-28b7-4462-96ed-be0cf7052ccc       [189637]  34586.0       34587.0   
3  9f0d82ae-38bf-45d8-8cda-982b6ead1d72       [435231]  65170.0       65171.0   
4  5aa12aff-db3c-4be5-822b-3afdf536b33e         [1278]      NaN           NaN   
5  ef027846-be81-4959-a6b5-56bd01b1e68a       [472997]  78170.0           NaN   

   tcgplayer_id  cardmarket_id              name lang  ...  \
0       14240.0        13850.0       Fury Sliver   en  ...   
1       33347.0        21851.0     Kor Outfitter   en  ...   
3      145764.

In [12]:
data.head()

,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,tcgplayer_etched_id,attraction_lights,color_indicator,variation_of,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,65171.0,145764.0,301766.0,Siren Lookout,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],NaN,NaN,1623.0,5664.0,Web,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,card,0001f1ef-b957-4a55-b47f-14839cdbab6f,ef027846-be81-4959-a6b5-56bd01b1e68a,[472997],78170.0,NaN,198861.0,400134.0,Venerable Knight,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# remove cards with duplicate names
data = data.drop_duplicates(subset=['name'])
print(data.shape)
data.head()

(25438, 84)


,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,tcgplayer_etched_id,attraction_lights,color_indicator,variation_of,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,65171.0,145764.0,301766.0,Siren Lookout,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],NaN,NaN,1623.0,5664.0,Web,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,card,0001f1ef-b957-4a55-b47f-14839cdbab6f,ef027846-be81-4959-a6b5-56bd01b1e68a,[472997],78170.0,NaN,198861.0,400134.0,Venerable Knight,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
print(data.shape)
data.head()

(25438, 84)


,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,tcgplayer_etched_id,attraction_lights,color_indicator,variation_of,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,65171.0,145764.0,301766.0,Siren Lookout,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],NaN,NaN,1623.0,5664.0,Web,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,card,0001f1ef-b957-4a55-b47f-14839cdbab6f,ef027846-be81-4959-a6b5-56bd01b1e68a,[472997],78170.0,NaN,198861.0,400134.0,Venerable Knight,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
data.drop(columns=['mtgo_id', 'mtgo_foil_id', 'arena_id', 'tcgplayer_id', 'cardmarket_id', 'legalities', 'games', 'reserved', 'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint', 'variation', 'artist', 'artist_ids', 'illustration_id', 'border_color', 'booster', 'story_spotlight', 'edhrec_rank', 'penny_rank','prices', 'promo_types', 'arena_id', 'preview', 'security_stamp', 'tcgplayer_etched_id', 'variation_of','released_at', 'set_id','set', 'set_uri', 'set_search_uri', 'scryfall_set_uri', 'rulings_uri', 'prints_search_uri', 'card_back_id', 'frame', 'related_uris'])


,object,id,oracle_id,multiverse_ids,name,lang,uri,scryfall_uri,layout,highres_image,...,printed_name,card_faces,attraction_lights,color_indicator,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],Fury Sliver,en,https://api.scryfall.com/cards/0000579f-7b35-4...,https://scryfall.com/card/tsp/157/fury-sliver?...,normal,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],Kor Outfitter,en,https://api.scryfall.com/cards/00006596-1166-4...,https://scryfall.com/card/zen/21/kor-outfitter...,normal,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],Siren Lookout,en,https://api.scryfall.com/cards/0000cd57-91fe-4...,https://scryfall.com/card/xln/78/siren-lookout...,normal,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],Web,en,https://api.scryfall.com/cards/00012bd8-ed68-4...,https://scryfall.com/card/3ed/229/web?utm_sour...,normal,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,card,0001f1ef-b957-4a55-b47f-14839cdbab6f,ef027846-be81-4959-a6b5-56bd01b1e68a,[472997],Venerable Knight,en,https://api.scryfall.com/cards/0001f1ef-b957-4...,https://scryfall.com/card/eld/35/venerable-kni...,normal,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80690,card,ffdfad1f-d316-4023-9c79-8c24f1cd31d0,7ddd84d2-1a6d-43f5-a42d-3c66eee5f7e5,[548297],Born to Drive,en,https://api.scryfall.com/cards/ffdfad1f-d316-4...,https://scryfall.com/card/neo/6/born-to-drive?...,normal,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80700,card,ffe4d34f-68f0-4d79-9aab-58c5304224d9,6dab0e1a-f39a-4113-9d13-b067cfefbdb3,[240069],Stern Mentor,en,https://api.scryfall.com/cards/ffe4d34f-68f0-4...,https://scryfall.com/card/avr/77/stern-mentor?...,normal,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80703,card,ffe93b27-f8ae-4abf-8ade-90f503f132c2,6d37ba4b-ff56-4eec-9dc2-2d7f357dc9c9,[506924],Youthful Valkyrie,en,https://api.scryfall.com/cards/ffe93b27-f8ae-4...,https://scryfall.com/card/khm/382/youthful-val...,normal,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80704,card,ffe9ee1c-5eb3-4d63-a641-0ec5adf7b058,e60b23ba-2258-4da1-b873-452ee215846b,[583793],Fallaji Vanguard,en,https://api.scryfall.com/cards/ffe9ee1c-5eb3-4...,https://scryfall.com/card/bro/210/fallaji-vang...,normal,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# now that we dont have any other languages, we can drop the lang column
data = data.drop(columns=['lang'])
print(data.shape)
data.head()

(25438, 83)


,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,released_at,...,tcgplayer_etched_id,attraction_lights,color_indicator,variation_of,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,2006-10-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,2009-10-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,65171.0,145764.0,301766.0,Siren Lookout,2017-09-29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],NaN,NaN,1623.0,5664.0,Web,1994-04-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,card,0001f1ef-b957-4a55-b47f-14839cdbab6f,ef027846-be81-4959-a6b5-56bd01b1e68a,[472997],78170.0,NaN,198861.0,400134.0,Venerable Knight,2019-10-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
data = data.drop(columns=['mtgo_id', 'mtgo_foil_id', 'arena_id', 'tcgplayer_id', 'cardmarket_id', 'legalities', 'games', 'reserved', 'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint', 'variation', 'artist', 'artist_ids', 'illustration_id', 'border_color', 'booster', 'story_spotlight', 'edhrec_rank', 'penny_rank','prices', 'promo_types', 'arena_id', 'preview', 'security_stamp', 'tcgplayer_etched_id', 'variation_of','released_at', 'set_id','set', 'set_uri', 'set_search_uri', 'scryfall_set_uri', 'rulings_uri', 'prints_search_uri', 'card_back_id', 'frame', 'related_uris'])

In [19]:
print(data.shape)
data.head()

(25438, 43)


,object,id,oracle_id,multiverse_ids,name,uri,scryfall_uri,layout,highres_image,image_status,...,printed_name,card_faces,attraction_lights,color_indicator,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],Fury Sliver,https://api.scryfall.com/cards/0000579f-7b35-4...,https://scryfall.com/card/tsp/157/fury-sliver?...,normal,True,highres_scan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],Kor Outfitter,https://api.scryfall.com/cards/00006596-1166-4...,https://scryfall.com/card/zen/21/kor-outfitter...,normal,True,highres_scan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],Siren Lookout,https://api.scryfall.com/cards/0000cd57-91fe-4...,https://scryfall.com/card/xln/78/siren-lookout...,normal,True,highres_scan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],Web,https://api.scryfall.com/cards/00012bd8-ed68-4...,https://scryfall.com/card/3ed/229/web?utm_sour...,normal,True,highres_scan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,card,0001f1ef-b957-4a55-b47f-14839cdbab6f,ef027846-be81-4959-a6b5-56bd01b1e68a,[472997],Venerable Knight,https://api.scryfall.com/cards/0001f1ef-b957-4...,https://scryfall.com/card/eld/35/venerable-kni...,normal,True,highres_scan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# rplace the card name in the oracle_text with the token <card_name>
data['oracle_text'] = data['oracle_text'].str.replace(data['name'], '<card_name>')

TypeError: unhashable type: 'Series'

In [21]:
data['oracle_text'] = data.apply(lambda x: x['oracle_text'].replace(x['name'], '<card_name>'), axis=1)


In [22]:
print(data.shape)
data.head()

(25438, 43)


,object,id,oracle_id,multiverse_ids,name,uri,scryfall_uri,layout,highres_image,image_status,...,printed_name,card_faces,attraction_lights,color_indicator,life_modifier,hand_modifier,printed_type_line,printed_text,content_warning,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],Fury Sliver,https://api.scryfall.com/cards/0000579f-7b35-4...,https://scryfall.com/card/tsp/157/fury-sliver?...,normal,True,highres_scan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],Kor Outfitter,https://api.scryfall.com/cards/00006596-1166-4...,https://scryfall.com/card/zen/21/kor-outfitter...,normal,True,highres_scan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],Siren Lookout,https://api.scryfall.com/cards/0000cd57-91fe-4...,https://scryfall.com/card/xln/78/siren-lookout...,normal,True,highres_scan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],Web,https://api.scryfall.com/cards/00012bd8-ed68-4...,https://scryfall.com/card/3ed/229/web?utm_sour...,normal,True,highres_scan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,card,0001f1ef-b957-4a55-b47f-14839cdbab6f,ef027846-be81-4959-a6b5-56bd01b1e68a,[472997],Venerable Knight,https://api.scryfall.com/cards/0001f1ef-b957-4...,https://scryfall.com/card/eld/35/venerable-kni...,normal,True,highres_scan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# Build a string representation of each card to be saved out as a text file.
# Card data will be represented as a string with the following format:
# "<|endoftext|>{card_name} | ?{mana_cost} | {type_line} | {rarity} | ?{oracle_text} | ?{flavor_text} | ?{power} | ?{toughness}<|endoftext|>"
# The ? indicates that the field may be empty.

# end of text token
eot = "<|endoftext|>"
card_data = []
for index, row in data.iterrows():
    card_data.append(f"{eot}{row['name']} | {row['mana_cost']} | {row['type_line']} | {row['rarity']} | {row['oracle_text']} | {row['flavor_text']} | {row['power']} | {row['toughness']}{eot}")

# sample the first 10 cards
card_data[:10]


['<|endoftext|>Fury Sliver | {5}{R} | Creature — Sliver | uncommon | All Sliver creatures have double strike. | "A rift opened, and our arrows were abruptly stilled. To move was to push the world. But the sliver\'s claw still twitched, red wounds appeared in Thed\'s chest, and ribbons of blood hung in the air."\n—Adom Capashen, Benalish hero | 3 | 3<|endoftext|>',
 '<|endoftext|>Kor Outfitter | {W}{W} | Creature — Kor Soldier | common | When <card_name> enters the battlefield, you may attach target Equipment you control to target creature you control. | "We take only what we need to survive. Believe me, you will need this." | 2 | 2<|endoftext|>',
 "<|endoftext|>Siren Lookout | {2}{U} | Creature — Siren Pirate | common | Flying\nWhen <card_name> enters the battlefield, it explores. (Reveal the top card of your library. Put that card into your hand if it's a land. Otherwise, put a +1/+1 counter on this creature, then put the card back or put it into your graveyard.) | nan | 1 | 2<|endoft

In [25]:
# save the card data to a text file
with open('data/mtg_char/mtg_card_data.txt', 'w') as f:
    f.write('\n'.join(card_data))

In [26]:
# write out the dataframe to a json file
data.to_json('data/mtg_char/mtg_card_data.json', orient='records', lines=True)